In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
from tqdm import tqdm
from tracking_utils import *
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [2]:
# SELECT USED MODEL
model = 'faster'

# SELECT SEQUENCE TO EVALUATE
S = 'seq1'

# SELECT CAMERA TO EVALUATE
C = 'c03'

# SET NUMBER OF FRAMES
num_frames = int(readFrameCount(S, C))

In [3]:
# Load GT detections
gt_detect_path = f'./cam_pred_gt/{S}_{C}_gt.pkl'

# Load computed detections
detection_path = f'./cam_pred/{S}_{model}_{C}.pkl'

with open(detection_path, 'rb') as f:
    all_detections = pkl.load(f)

with open(gt_detect_path, 'rb') as f:
    all_gt_detections = pkl.load(f)

## Box intersection tracking

In [4]:
def update_track3(detections_pd, previous_detections_pd, tolerance = 0.5, of = None, imgs = None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)

    previous_tracks = []

    for index, previous_detection in previous_detections_pd.iterrows():
        # Calculating IoUs
        IoUlist = []

        length, _ = detections_pd.shape

        for i in range(length):
            IoU = previous_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)

        indexMax = IoUlist.index(max(IoUlist))

        # Updating detection tracks based on best IoU matches
        if max(IoUlist) > tolerance and detections_pd.at[indexMax, 'updated'] != True:
            detections_pd.at[indexMax, 'track'] = previous_detection['track']
            detections_pd.at[indexMax, 'colour'] = previous_detection['colour']
            for line in previous_detection['line']:
                if line != detections_pd.at[indexMax,'line'][0]:
                    detections_pd.at[indexMax,'line'].append(line)
            detections_pd.at[indexMax, 'updated'] = True

        previous_tracks.append(previous_detection['track'])

    # Create new tracks for unmatched detections
    idxs = detections_pd.index[detections_pd['updated'] == False].tolist()
    track_check = max(previous_tracks)+1
    for ind in idxs:
        detections_pd.at[ind, 'track'] = track_check
        track_check += 1


    # # Create new tracks for unmatched detections
    # idxs = detections_pd.index[detections_pd['updated'] == False].tolist()
    # track_check = 1
    # for ind in idxs:
    #     while track_check in previous_tracks:
    #         track_check += 1
    #     detections_pd.at[ind, 'track'] = track_check
    #     previous_tracks.append(track_check)
            
    return detections_pd

In [5]:
detection_history = []

# Get the first frame
detections_prev_pd = get_detection_dataframe(all_detections['0'], firstFrame = True)

# Pre-process the first frame if needed
#detections_prev_pd = remove_overlaps(detections_prev_pd, 0.9)
detection_history.append(detections_prev_pd)

past_frame = 0

for frame in tqdm(range(1, num_frames)):
    current_frame = frame

    detections_pd = get_detection_dataframe(all_detections[str(frame)])
    #detections_pd = remove_overlaps(detections_pd, 0.9)
    detections_pd = update_track3(detections_pd, detections_prev_pd, tolerance=0.2, imgs=[past_frame, current_frame])
    detection_history.append(detections_pd)
    detections_prev_pd = detections_pd

    past_frame = current_frame

pkl.dump(detection_history, open(f'./tracking_results/tracking_history_{S}_{C}.pkl', 'wb'))

100%|██████████| 1995/1995 [00:38<00:00, 52.44it/s] 


In [6]:
detections = {}
with open(f'./tracking_results/tracking_history_{S}_{C}.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Kalman

In [7]:
# Load video frames
results = {}

mot_tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3) # Sort Kalman tracker with default values

# Iterate Frames
for frame in tqdm(range(num_frames), desc = "Tracking objects each frame..."):
    dets = []

    detections = get_detection_dataframe(all_detections[str(frame)], iclLineAndUpdate = False, firstFrame = True).sort_values("track") # Load detections

    for (track_id, det, bbox, size, colour) in detections.itertuples(index=False): # Iter All Detections
        dets.append(np.array(bbox))

    trackers = mot_tracker.update(np.array(dets)) # Update tracker with current detections

    for d in trackers: # Store new bboxes
        d = d.astype(np.int32)
        if frame not in results:
            results[frame] = {d[4] % 100: {"bbox": [d[0], d[1], d[2], d[3]]}}
        else:
            results[frame][d[4] % 100] = {"bbox": [d[0], d[1], d[2], d[3]]}

            
# Save Results to Disk
pkl.dump(results, open(f"./tracking_results/sort_bbox_{S}_{C}.pkl", "wb"))

Tracking objects each frame...: 100%|██████████| 1996/1996 [00:09<00:00, 214.12it/s]


In [8]:
# Pkl to pd.DataFrame
colours = np.random.rand(100, 3) #Generate Random Colors
old_pkl = pkl.load(open(f"./tracking_results/sort_bbox_{S}_{C}.pkl", "rb"))
new_pkl = []
df = None

for frame in old_pkl.keys():
    new_data = {"frame": [],"track": [], "bbox": [], "colour": [], "size": [], "detection": []}
    old_data = old_pkl[frame]
    
    for track_id in old_data.keys():
        new_data["track"].append(track_id)
        bbox = old_data[track_id]["bbox"]
        new_data["bbox"].append(bbox)
        new_data["frame"].append("Frame {}".format(frame))
        new_data["colour"].append(np.round(colours[track_id]*255).astype(np.uint8))
        new_data["size"].append(int(np.abs(bbox[0] - bbox[2]) * np.abs(bbox[1] - bbox[3])))
        new_data["detection"].append(VehicleDetection(frame=frame, ID=track_id, width=np.abs(bbox[0] - bbox[2]), conf=0.5,
                                                      height=np.abs(bbox[1] - bbox[3]), left=bbox[0], right=bbox[2], top=bbox[1], bot=bbox[3]))
    
    df = pd.DataFrame.from_dict(new_data).sort_values(by="track")

    new_pkl.append(df)

pkl.dump(new_pkl, open(f"./tracking_results/kalman_tracking_{S}_{C}.pkl", "wb"))

In [9]:
detections = {}
with open(f'./tracking_results/kalman_tracking_{S}_{C}.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Evaluation

In [10]:
total_frames = num_frames
initial_frame = 0

### Post-processing on tracked detections

In [11]:
# Checking minimum width and height of ground truth box detections
minW_gt = np.inf
minH_gt = np.inf

for frame in all_gt_detections:
    for detection in all_gt_detections[frame]:
        if detection.w < minW_gt:
            minW_gt = detection.w
        if detection.h < minH_gt:
            minH_gt = detection.h

# Removing detections that are under these minimum values (minus an optional margin)
i = initial_frame

minW = 1.00*minW_gt
minH = 1.00*minH_gt

while i < total_frames:
    for index, detection in detections[i].iterrows():
        indxs_to_erase = []
        if detection['detection'].w < minW and detection['detection'].h < minH:
            indxs_to_erase.append(index)
        detections[i].drop(indxs_to_erase, inplace = True)
    detections[i] = detections[i].reset_index(drop=True)
    i += 1

In [12]:
# Discarding box detections that are in contact with the video's edges
video_path = '../../AICity_data/train/S01/c003/vdo.avi' #MAKE ADAPTIVE
vidcap = cv2.VideoCapture(video_path)
_, image = vidcap.read()

i = initial_frame
while i < total_frames:
    for index, detection in detections[i].iterrows():
        indxs_to_erase = []
        if detection['detection'].xtl < 15 or detection['detection'].xbr > image.shape[1]-15 or detection['detection'].ytl < 15 or detection['detection'].ybr > image.shape[0]-15:
            indxs_to_erase.append(index)
        detections[i].drop(indxs_to_erase, inplace = True)
    detections[i] = detections[i].reset_index(drop=True)
    i += 1

In [13]:
# Discarding tracks with less than 5 frames of duration
min_frames = 5

i = initial_frame + 1

detections[0]['life'] = 1

while i < total_frames:
    detections[i]['life'] = 1
    length, _ = detections[i].shape
    
    for j in range(length):

        continuity = False

        track = detections[i].iloc[j]['track']

        for index_prev, detection_prev in detections[i-1].iterrows():
            if track == detection_prev['track']:
                detections[i].at[j, 'life'] = detection_prev['life'] + 1

        try:
            for index_next, detection_next in detections[i+1].iterrows():
                if track == detection_next['track']:
                    continuity = True
        except:
            pass

        if continuity == False and detections[i].iloc[j]['life'] < min_frames:
            a = detections[i].iloc[j]['life']-1

            while a >= 0:
                track_index = detections[i-int(a)].index[detections[i-int(a)]['track'] == track].tolist()
                detections[i-int(a)].at[track_index, 'life'] = -1
                a -= 1

    detections[i] = detections[i][detections[i]['life'] != -1]

    # indexes_to_drop = detections[i].index[detections[i]['life'] == -1].tolist()
    # detections[i].drop(indexes_to_drop, inplace = True)
    # detections[i] = detections[i].reset_index(drop=True)

    i += 1

In [14]:
# Discarding tracks that remain static throughout the entire sequence
tracks_to_discard = []

for detection in detections[total_frames-1].iterrows():
    track = detection[1]['track']
    detection_final = detection[1]['detection']

    if track in detections[0]['track']:
        detection_initial = detections[0][detections[0]['track'] == track]['detection']
        IoU_with_first_instance = detection_initial.IoU(detection_final)
        if IoU_with_first_instance > 0.5:
            tracks_to_discard.append(track)


# IoU = previous_detection['detection'].IoU(detections_pd.iloc[i]['detection'])

In [15]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames)):
    detections[frame] = detections[frame][detections[frame]['life'] != -1]
    for track_to_discard in tracks_to_discard:
        detections[frame] = detections[frame][detections[frame]['track'] != track_to_discard]
    acc, frame_id = tracking_acc(frame, all_gt_detections, detections, acc, max_iou=1)

print(f'{S} {C} results:')
display_metrics(acc)

100%|██████████| 1996/1996 [00:19<00:00, 100.98it/s]

seq1 c03 results:
     num_frames  precision    recall       idp       idr      idf1
acc        1996   0.866364  0.857872  0.739328  0.732081  0.735686


## GIF generation

In [16]:
video_path = '../../AICity_data/train/S01/c003/vdo.avi'

In [17]:
# SHOW GROUND TRUTH
s_gt = True

# SHOW DETECTIONS
s_detections = False

# SHOW TRACKS
s_tracks = True

In [18]:
# Generating a tracking GIF for a frame interval
gif1 = True
initial_gif_frame = 0 #700
final_gif_frame = 400 #750
scaling = 0.5

ims = []

vidcap = cv2.VideoCapture(video_path)
vidcap.set(1,initial_gif_frame)

if gif1:
    _, initial_image = vidcap.read()

    for i in tqdm(range(initial_gif_frame+1, final_gif_frame)):
        _, image = vidcap.read()

        detections_pd = get_detection_dataframe(all_detections[str(i)])

        gt_detections_pd = get_detection_dataframe(all_gt_detections[str(i+1)])

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Plot ground truth in blue
        if s_gt == True:
            for index, row in gt_detections_pd.iterrows():
                image = drawTrackingOnImage(image, row['bbox'], colour=(0, 0, 255), showTracking = False)

        # Plot current detections in green
        if s_detections == True:
            for index, row in detections_pd.iterrows():
                image = drawTrackingOnImage(image, row['bbox'], colour=(0, 255, 0), showTracking = False)

        # Plot tracks in different colors
        if s_tracks == True:
            for index, row in detections[i].iterrows():
                image = drawTrackingOnImage(image, row['bbox'], track=row['track'], colour=row['colour'])
            
        image = cv2.resize(image, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)

        ims.append(image)

    imageio.mimsave('tracking1.gif', ims, fps=20, duration = 0.3)

100%|██████████| 399/399 [00:04<00:00, 89.79it/s] 
